In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMRegressor

In [0]:
import datetime as dt

In [0]:
big_df =  pd.read_csv('dataset/data_new.csv')

In [4]:
big_df.dtypes

Origin            object
Destination       object
Dept_Date         object
Dept_Time         object
Arr_Time          object
Total_Fare       float64
Base_Fare        float64
Fuel_Fare        float64
Airways           object
Available        float64
Duration          object
Class_Type        object
Flight Number     object
Flight Code       object
FlightID          object
Hopping           object
Taken             object
dtype: object

In [0]:
def weekDay(dayVal):
    day, month, year = (int(x) for x in dayVal.split('-'))    
    ans = dt.date(year, month, day).strftime("%a")
    return ans
def weekDay2(dayVal):
    month , day, year = (int(x) for x in dayVal.split('/'))    
    ans = dt.date(year, month, day).strftime("%a")
    return ans

In [0]:
big_df['Taken_Day'] = big_df['Taken'].apply(weekDay)
big_df['Dept_Day'] = big_df['Dept_Date'].apply(weekDay2)

In [0]:
big_df["Dept_Date"] = big_df["Dept_Date"].apply(pd.to_datetime)
big_df["Taken"] = big_df["Taken"].apply(pd.to_datetime)

In [0]:
big_df['Days_To_Dept'] = (big_df['Dept_Date'] - big_df['Taken']).dt.days

In [0]:
big_df=big_df.drop(['Dept_Date'], axis=1)

In [0]:
big_df=big_df.drop(['Taken'], axis=1)

In [11]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51734 entries, 0 to 51733
Data columns (total 18 columns):
Origin           51734 non-null object
Destination      51734 non-null object
Dept_Time        51719 non-null object
Arr_Time         51719 non-null object
Total_Fare       51719 non-null float64
Base_Fare        51719 non-null float64
Fuel_Fare        51719 non-null float64
Airways          51719 non-null object
Available        51719 non-null float64
Duration         51719 non-null object
Class_Type       51719 non-null object
Flight Number    51719 non-null object
Flight Code      51719 non-null object
FlightID         51719 non-null object
Hopping          51719 non-null object
Taken_Day        51734 non-null object
Dept_Day         51734 non-null object
Days_To_Dept     51734 non-null int64
dtypes: float64(4), int64(1), object(13)
memory usage: 7.1+ MB


In [0]:
big_df['Dep_Hour'] = big_df['Dept_Time'] .str.split(':').str[0]
big_df['Dep_Hour'] = pd.to_numeric(big_df['Dep_Hour'], errors='coerce')
big_df= big_df.dropna(subset=['Dep_Hour'])
big_df['Dep_Hour'] = big_df['Dep_Hour'].astype(int)

In [0]:
big_df=big_df.drop(['Dept_Time'], axis=1)

In [0]:
big_df['Total_Fare'].fillna((big_df['Total_Fare'].mean()), inplace=True)

In [15]:
big_df.describe()

,Total_Fare,Base_Fare,Fuel_Fare,Available,Days_To_Dept,Dep_Hour
count,51719.000000,51719.000000,51719.000000,51719.000000,51719.000000,51719.000000
mean,7910.773739,6212.169493,1280.208589,8.921847,45.286529,11.214583
std,4069.358744,3648.616760,847.002890,10.725873,26.229297,6.134450
min,3211.000000,1530.000000,218.000000,1.000000,0.000000,0.000000
25%,5223.000000,3800.000000,481.000000,5.000000,23.000000,6.000000
50%,7009.000000,5350.000000,860.000000,7.000000,46.000000,9.000000
75%,9355.000000,7550.000000,2171.000000,9.000000,68.000000,17.000000
max,54826.000000,49906.000000,4398.000000,112.000000,92.000000,23.000000


In [0]:
import re
def convertToDecimal(value):
    h, m = re.sub("\D"," ",value).split()
    return int(h)*60+int(m)

In [0]:
big_df.loc[:,'Duration'] = big_df.loc[:,'Duration'].apply(convertToDecimal)

In [0]:
def assign_session(val):
    if val > 0.0 and val <= 6.0:
        return 'Night'
    elif val > 6.0 and val <= 12.0:
        return 'Morning'
    elif val > 12.0 and val <= 18.0:
        return 'Afternoon'
    else:
        return 'Evening'

In [0]:
big_df['Session'] = big_df['Dep_Hour'].apply(assign_session)

In [20]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51719 entries, 0 to 51733
Data columns (total 19 columns):
Origin           51719 non-null object
Destination      51719 non-null object
Arr_Time         51719 non-null object
Total_Fare       51719 non-null float64
Base_Fare        51719 non-null float64
Fuel_Fare        51719 non-null float64
Airways          51719 non-null object
Available        51719 non-null float64
Duration         51719 non-null int64
Class_Type       51719 non-null object
Flight Number    51719 non-null object
Flight Code      51719 non-null object
FlightID         51719 non-null object
Hopping          51719 non-null object
Taken_Day        51719 non-null object
Dept_Day         51719 non-null object
Days_To_Dept     51719 non-null int64
Dep_Hour         51719 non-null int64
Session          51719 non-null object
dtypes: float64(4), int64(3), object(12)
memory usage: 7.9+ MB


In [21]:
big_df.head()

,Origin,Destination,Arr_Time,Total_Fare,Base_Fare,Fuel_Fare,Airways,Available,Duration,Class_Type,Flight Number,Flight Code,FlightID,Hopping,Taken_Day,Dept_Day,Days_To_Dept,Dep_Hour,Session
0,BLR,DEL,2:20,7517.0,6450.0,796.0,IndiGo,1.0,170,Economy,2484,6E,6E_2484_,False,Tue,Tue,0,23,Evening
1,BLR,DEL,12:30,7806.0,7025.0,460.0,Spicejet,9.0,175,Economy,192,SG,SG_192_,False,Tue,Tue,0,9,Morning
2,BLR,DEL,21:35,7888.0,6755.0,862.0,IndiGo,13.0,285,Economy,427_ 202,6E,6E_427_ 202_,False,Tue,Tue,0,16,Afternoon
3,BLR,DEL,2:10,8009.0,7195.0,661.0,AirAsia,11.0,165,Economy,722,I5,I5_722_,False,Tue,Tue,0,23,Evening
4,BLR,DEL,19:45,8041.0,6950.0,820.0,IndiGo,3.0,165,Economy,2988,6E,6E_2988_,False,Tue,Tue,0,17,Afternoon


In [0]:
big_df.loc[:,'FlightGroup'] =big_df.loc[:,['Airways','Session']].astype(str).apply('_'.join,1)

In [0]:
big_df = big_df.loc[big_df['Destination']=='DEL']

In [24]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20816 entries, 0 to 51720
Data columns (total 20 columns):
Origin           20816 non-null object
Destination      20816 non-null object
Arr_Time         20816 non-null object
Total_Fare       20816 non-null float64
Base_Fare        20816 non-null float64
Fuel_Fare        20816 non-null float64
Airways          20816 non-null object
Available        20816 non-null float64
Duration         20816 non-null int64
Class_Type       20816 non-null object
Flight Number    20816 non-null object
Flight Code      20816 non-null object
FlightID         20816 non-null object
Hopping          20816 non-null object
Taken_Day        20816 non-null object
Dept_Day         20816 non-null object
Days_To_Dept     20816 non-null int64
Dep_Hour         20816 non-null int64
Session          20816 non-null object
FlightGroup      20816 non-null object
dtypes: float64(4), int64(3), object(13)
memory usage: 3.3+ MB


In [25]:
flight_group_unique = big_df.FlightGroup.unique()
dept_day_unique = big_df.Dept_Day.unique()
taken_day_unique = big_df.Taken_Day.unique()
day_to_dept_unique = big_df.Days_To_Dept.unique()
duration_unique = big_df.Duration.unique()
class_type_unique = big_df.Class_Type.unique()
hopping_unique = big_df.Hopping.unique()
print(flight_group_unique,'\n')
print(dept_day_unique,'\n')
print(taken_day_unique,'\n')
print(day_to_dept_unique,'\n')
print(duration_unique,'\n')
print(class_type_unique,'\n')
print(hopping_unique,'\n')

['IndiGo_Evening' 'Spicejet_Morning' 'IndiGo_Afternoon' 'AirAsia_Evening'
 'Spicejet_Evening' 'IndiGo_Morning' 'Air India_Morning'
 'Air India_Evening' 'Air India_Afternoon' 'Air India_Night'
 'Jet Airways_Afternoon' 'Jet Airways_Morning' 'Vistara_Morning'
 'Vistara_Evening' 'AirAsia_Night' 'Spicejet_Night' 'Jet Airways_Evening'
 'IndiGo_Night' 'Jet Airways_Night' 'Spicejet_Afternoon'
 'Spicejet HBag Only_Night' 'Spicejet HBag Only_Evening'
 'Spicejet HBag Only_Morning'] 

['Tue' 'Wed' 'Thu' 'Fri' 'Sat' 'Sun' 'Mon'] 

['Tue' 'Wed' 'Thu' 'Fri' 'Sat' 'Sun'] 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92] 

[170 175 285 165 180 160 295 155 400 275 485 260 245 280 290] 

['Economy'] 

[False True] 



In [0]:
from sklearn.preprocessing import LabelEncoder

lb_encode = LabelEncoder()
big_df["Hopping"] = lb_encode.fit_transform(big_df["Hopping"])
big_df["Airways"] = lb_encode.fit_transform(big_df["Airways"])
#big_df["Destination"] = lb_encode.fit_transform(big_df["Destination"])
#big_df["Origin"] = lb_encode.fit_transform(big_df["Origin"])
big_df["FlightGroup"] = lb_encode.fit_transform(big_df["FlightGroup"])
big_df["Session"] = lb_encode.fit_transform(big_df["Session"])

In [0]:
big_df["Taken_Day"] = lb_encode.fit_transform(big_df["Taken_Day"])
big_df["Dept_Day"] = lb_encode.fit_transform(big_df["Dept_Day"])

In [28]:
big_df.head()

,Origin,Destination,Arr_Time,Total_Fare,Base_Fare,Fuel_Fare,Airways,Available,Duration,Class_Type,Flight Number,Flight Code,FlightID,Hopping,Taken_Day,Dept_Day,Days_To_Dept,Dep_Hour,Session,FlightGroup
0,BLR,DEL,2:20,7517.0,6450.0,796.0,2,1.0,170,Economy,2484,6E,6E_2484_,0,4,5,0,23,1,7
1,BLR,DEL,12:30,7806.0,7025.0,460.0,4,9.0,175,Economy,192,SG,SG_192_,0,4,5,0,9,2,19
2,BLR,DEL,21:35,7888.0,6755.0,862.0,2,13.0,285,Economy,427_ 202,6E,6E_427_ 202_,0,4,5,0,16,0,6
3,BLR,DEL,2:10,8009.0,7195.0,661.0,1,11.0,165,Economy,722,I5,I5_722_,0,4,5,0,23,1,4
4,BLR,DEL,19:45,8041.0,6950.0,820.0,2,3.0,165,Economy,2988,6E,6E_2988_,0,4,5,0,17,0,6


In [0]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns  

In [30]:
missing_values_table(big_df)

Your selected dataframe has 20 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [31]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20816 entries, 0 to 51720
Data columns (total 20 columns):
Origin           20816 non-null object
Destination      20816 non-null object
Arr_Time         20816 non-null object
Total_Fare       20816 non-null float64
Base_Fare        20816 non-null float64
Fuel_Fare        20816 non-null float64
Airways          20816 non-null int64
Available        20816 non-null float64
Duration         20816 non-null int64
Class_Type       20816 non-null object
Flight Number    20816 non-null object
Flight Code      20816 non-null object
FlightID         20816 non-null object
Hopping          20816 non-null int64
Taken_Day        20816 non-null int64
Dept_Day         20816 non-null int64
Days_To_Dept     20816 non-null int64
Dep_Hour         20816 non-null int64
Session          20816 non-null int64
FlightGroup      20816 non-null int64
dtypes: float64(4), int64(9), object(7)
memory usage: 3.3+ MB


In [0]:
big_df.drop(columns=['Base_Fare','Fuel_Fare','Arr_Time','Available','Class_Type','Flight Number','Flight Code','FlightID','Dep_Hour'],axis=1,inplace=True)

In [0]:
big_df.drop(columns=['Origin','Destination'],axis=1,inplace=True)

In [34]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20816 entries, 0 to 51720
Data columns (total 9 columns):
Total_Fare      20816 non-null float64
Airways         20816 non-null int64
Duration        20816 non-null int64
Hopping         20816 non-null int64
Taken_Day       20816 non-null int64
Dept_Day        20816 non-null int64
Days_To_Dept    20816 non-null int64
Session         20816 non-null int64
FlightGroup     20816 non-null int64
dtypes: float64(1), int64(8)
memory usage: 1.6 MB


In [35]:
big_df.head()

,Total_Fare,Airways,Duration,Hopping,Taken_Day,Dept_Day,Days_To_Dept,Session,FlightGroup
0,7517.0,2,170,0,4,5,0,1,7
1,7806.0,4,175,0,4,5,0,2,19
2,7888.0,2,285,0,4,5,0,0,6
3,8009.0,1,165,0,4,5,0,1,4
4,8041.0,2,165,0,4,5,0,0,6


In [36]:
df_train = big_df[0:18683]
df_test = big_df[18683:]
df_test = df_test.drop(['Total_Fare'], axis =1)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18683 entries, 0 to 46243
Data columns (total 9 columns):
Total_Fare      18683 non-null float64
Airways         18683 non-null int64
Duration        18683 non-null int64
Hopping         18683 non-null int64
Taken_Day       18683 non-null int64
Dept_Day        18683 non-null int64
Days_To_Dept    18683 non-null int64
Session         18683 non-null int64
FlightGroup     18683 non-null int64
dtypes: float64(1), int64(8)
memory usage: 1.4 MB


In [0]:
X = df_train.drop(axis=1,columns=['Total_Fare'])
y = df_train.Total_Fare

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [39]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url = "http://i.imgur.com/QBuDOjs.jpg")

In [0]:
import xgboost as xgb
from xgboost import XGBRegressor

In [0]:
from sklearn.model_selection import GridSearchCV
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
get_ipython().run_line_magic('matplotlib', 'inline')
import xgboost as xgb
from xgboost import XGBRegressor

In [0]:
'''def modelfit(alg, dtrain, target, useTrainCV=True, 
             cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.values, 
                              label=y.values)
        
        print("\nGetting Cross-validation result..")
        cvresult = xgb.cv(xgb_param, xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds,metrics='rmse', 
                          early_stopping_rounds=early_stopping_rounds,
                          verbose_eval = True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    print("\nFitting algorithm to data...")
    alg.fit(dtrain, target, eval_metric='rmse')
        
    #Predict training set:
    print("\nPredicting from training data...")
    dtrain_predictions = alg.predict(dtrain)
        
    #Print model report:
    print("\nModel Report")
    print("RMSE : %.4g" % np.sqrt(mean_squared_error(target.values,
                                             dtrain_predictions)))'''

In [0]:
#Build our cross validation method
kfolds = KFold(n_splits=50,shuffle=True, random_state=100)

def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, X, y, 
                                   scoring="neg_mean_squared_error", 
                                   cv = kfolds))
    return(rmse)

In [44]:
xgb3 = XGBRegressor(learning_rate =0.1, n_estimators=200, max_depth=10,
                     min_child_weight=5 ,gamma=0, subsample=0.7,max_bin=20,
                     colsample_bytree=0.8,objective= 'reg:linear',
                     nthread=4,scale_pos_weight=1,seed=27, reg_alpha=0.00006)

xgb_fit = xgb3.fit(X_train, y_train)

cv_rmse(xgb_fit).mean()

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

958.9030823033462

In [0]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.pipeline import make_pipeline

In [46]:
from lightgbm import LGBMRegressor

lgbm_model = LGBMRegressor(objective='regression',num_leaves=31,
                              learning_rate=0.1, n_estimators=200,
                              max_bin = 100, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.8,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =20, min_sum_hessian_in_leaf = 11)
lgbm_fit = lgbm_model.fit(X_train, y_train)

cv_rmse(lgbm_fit).mean()

987.3978678674819

In [47]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=300, random_state=42)
forest_fit = forest.fit(X_train,y_train)
cv_rmse(forest_fit).mean()

995.393931958713

In [0]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.pipeline import make_pipeline

#setup models

lightgbm = make_pipeline(RobustScaler(),
                        LGBMRegressor(objective='regression',num_leaves=31,
                              learning_rate=0.1, n_estimators=200,
                              max_bin = 100, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.8,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11))

xgboost = make_pipeline(RobustScaler(),
                        XGBRegressor(learning_rate =0.1, n_estimators=200, max_depth=10,
                     min_child_weight=5 ,gamma=0, subsample=0.7,
                     colsample_bytree=0.8,objective= 'reg:linear',
                     nthread=4,scale_pos_weight=1,seed=27, reg_alpha=0.00006))

forest = make_pipeline(RobustScaler(),
                        RandomForestRegressor(n_estimators=300, random_state=42))
 

#stack
stack_gen = StackingCVRegressor(regressors=( lightgbm,xgboost,forest), 
                               meta_regressor=xgboost,
                               use_features_in_secondary=True)

#prepare dataframes
stackX = np.array(X_train)
stacky = np.array(y_train)

In [0]:
stack_gen_model = stack_gen.fit(stackX, stacky)

In [0]:
stack_gen_preds = stack_gen_model.predict(X_test)
xgb_preds = xgb_fit.predict(X_test)
lgbm_preds = lgbm_fit.predict(X_test)
forest_preds = forest_fit.predict(X_test)

In [51]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse_forest = np.sqrt(mean_squared_error(y_test, forest_preds))
print("RMSE_FOREST: %f" % (rmse_forest))

rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_preds))
print("RMSE_XGB: %f" % (rmse_xgb))

rmse_lgbm = np.sqrt(mean_squared_error(y_test, lgbm_preds))
print("RMSE_LGBM: %f" % (rmse_lgbm))

rmse_stack = np.sqrt(mean_squared_error(y_test, stack_gen_preds))
print("RMSE_STACK: %f" % (rmse_stack))

RMSE_FOREST: 1095.348417
RMSE_XGB: 1064.744983
RMSE_LGBM: 1109.014249
RMSE_STACK: 1088.892263


In [52]:
from sklearn.metrics import r2_score
print('r2_xgb' ,r2_score(y_test,xgb_preds) )
print('r2_lgbm' , r2_score(y_test,lgbm_preds) )
print('r2_forest' , r2_score(y_test,forest_preds) )
print('r2_stack' , r2_score(y_test,stack_gen_preds) )

r2_xgb 0.8376459342144456
r2_lgbm 0.8238647760832406
r2_forest 0.8281788846960183
r2_stack 0.8301983962660315


In [0]:
abs_error = abs(stack_gen_preds - y_test)
#print('Absolute Error',abs_error)
mabe = 100 * (abs_error / y_test)

In [54]:
accuracy = 100 - np.mean(mabe)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 92.52 %.


In [55]:
testval = X_test.iloc[0].to_frame().T
print(testval)

       Airways  Duration  Hopping  Taken_Day  Dept_Day  Days_To_Dept  Session  \
29846        2       170        0          3         3            80        1   

       FlightGroup  
29846            7  


In [56]:
stack_gen_model.predict(testval)

array([3423.7727], dtype=float32)

In [59]:
print(y_test.iloc[0])

3417.0


In [0]:
rf_filename = 'stacked_model.pkl'
import pickle as pkl
pkl.dump(forest, open(rf_filename, 'wb'))